## Install rapidsai and *cuml*

In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 355, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 355 (delta 68), reused 51 (delta 51), pack-reused 269
Receiving objects: 100% (355/355), 97.77 KiB | 24.44 MiB/s, done.
Resolving deltas: 100% (170/170), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 1.4 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/env-check.py", line 26, in <module>
    gpu_name = pynvml.nvmlDeviceGetName(pynvml.nvmlDeviceGetHandleByIndex(0)).decode('UTF-8')
AttributeError: 'str' object has no attribute 'decode'


In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,542 kB]
Hit:13 http://ppa.launchpad

In [ ]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...


In [ ]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'
!pip uninstall cupy -y

Found existing installation: cffi 1.15.1
Uninstalling cffi-1.15.1:
  Successfully uninstalled cffi-1.15.1
Found existing installation: cryptography 39.0.2
Uninstalling cryptography-39.0.2:
  Successfully uninstalled cryptography-39.0.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.3/444.3 kB 14.5 MB/s eta 0:00:00
Installing RAPIDS Stable 22.12
Starting the RAPIDS install on Colab.  This will take about 15 minutes.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... WARNING conda.core.solve:_add_specs(640): pinned spec python=3.9 conflicts with explicit specs.  Overriding pinned spec.
WARNING conda.core.solve:_add_specs(640): pinned spec cudatoolkit=11.8 conflicts with explicit specs.  Overriding pinned spec.
failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 

In [ ]:
!conda install_rapids cuml


CommandNotFoundError: No command 'conda install_rapids'.
Did you mean 'conda install'?



In [ ]:
import cuml
from cuml.manifold import UMAP

ModuleNotFoundError: ignored

## File for modeling

In [1]:
!pip3 install bertopic
import numpy as np
import pandas as pd
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 78.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
    

In [6]:
# Load data and check properties

transcripts = pd.read_csv('/content/drive/MyDrive/sports_word_256.csv.gz', compression='gzip', usecols=[2])
print(transcripts.shape)
transcripts.head(2)

(368835, 1)


,transcript_subset
0,Hello and welcome to the law review podcast. M...
1,"as well as several other changes, so we'll get..."


In [7]:
# Create list of documents as input for enbeddings
# without additional sampling
docs = transcripts.transcript_subset.to_list()
print(len(docs))

# With additional sampling
# sample_docs = transcripts.transcript_subset.sample(1000000, random_state =42).to_list()
# print(len(sample_docs))

368835


In [ ]:
# load embeddings
embeddings_256 = np.load('/content/drive/MyDrive/embeddings_256.npy')

In [ ]:
# load t-SNE dim reduced embeddings
tsne_embeddings = np.load('/content/drive/MyDrive/tsne_dimred_embeddings.npy')
tsne_embeddings.shape

(343928, 3)

## Google colab stuff

In [2]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')  # This will prompt for authorization

Mounted at /content/drive


In [3]:
# Check memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print(f'Your runtime has {ram_gb:.1f} gigabytes of available RAM\n')

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
# Check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv


## BERT v1
**Modelling data on batch level. Input instance size = max_sequence_length of embedding model**
all-MiniLM-L6-v2 max_sequence_length: 256


In [ ]:
# Define submodels
#from umap.parametric_umap import ParametricUMAP
#from sklearn.decomposition import PCA
#from sklearn.decomposition import KernelPCA
from bertopic.dimensionality import BaseDimensionalityReduction

#dim_model = PCA(n_components = 100, random_state=42)
#pca_kernel = KernelPCA(n_components=5, kernel='linear')

#sentence_model = SentenceTransformer("all-MiniLM-L6-v2")


dim_model = BaseDimensionalityReduction()

#umap_model = UMAP(
#    n_neighbors=15,
 #   n_components=5, 
  #  min_dist=0.0, 
   # metric='cosine',
    #low_memory=True)

hdbscan_model = HDBSCAN(
    min_cluster_size = 15,
    metric='euclidean', # same as cosine for normalised data
    cluster_selection_method='eom', 
    prediction_data=True)

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

vectorizer_model = CountVectorizer(stop_words='english')

In [ ]:
# Initialize BERTopic and run

topic_model = BERTopic(
    #embedding_model=sentence_model,
    umap_model=dim_model,
    hdbscan_model=hdbscan_model,
    ctfidf_model=ctfidf_model,
    vectorizer_model = vectorizer_model,
    calculate_probabilities=True,
    low_memory=True, 
    verbose=True  # progress bar
    )

topics, probs = topic_model.fit_transform(docs, tsne_embeddings)

2023-03-15 10:02:25,179 - BERTopic - The dimensionality reduction algorithm did not contain the `y` parameter and therefore the `y` parameter was not used
2023-03-15 10:02:25,181 - BERTopic - Reduced dimensionality


In [ ]:
topic_model.save('/content/drive/MyDrive/BERT_v2_proba')

In [ ]:
len(topic_model.get_topics())

1855

## BERTtopic V2
**Modelling data on sentence level. Input instances size = 1 sentence**

In [ ]:
sentence_25 = pd.read_csv('sentences_chunkssize_25.csv.gz', usecols=[1,2,3], compression='gzip')
print(sentence_25.shape)
sentence_25.head(2)

(215064, 3)


,episode_id,transcript_subset,sentence_enumerated
0,7tYqM5F5SKtt7lFgcimgAh,I'm Daniel Williams director of active chicks ...,0 - 25
1,7tYqM5F5SKtt7lFgcimgAh,"And when I say, you know, I'm making decisions...",25 - 50


In [ ]:
# Create list of documents as input for BERTopic
docs_sentences = sentence_25.transcript.to_list()
len(docs_sentences)

215064

In [ ]:
# Define submodels
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

umap_model = UMAP(
    n_neighbors=15, 
    n_components=5, 
    min_dist=0.0, 
    metric='cosine')

hdbscan_model = HDBSCAN(
    min_cluster_size=len(docs)*0.025, # Limit at 400 clusters
    metric='euclidean', 
    cluster_selection_method='eom', 
    prediction_data=True)

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

In [ ]:
# Initialize BERTopic and run

bert_v2 = BERTopic(
    embedding_model=sentence_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    ctfidf_model=ctfidf_model,
    verbose=True
    )

topics, probs = bert_v2.fit_transform(docs_sentences)

# save model
bert_v2.save('BERT_v2')

Batches:   0%|          | 0/6721 [00:00<?, ?it/s]

## Embeddings

In [11]:
# Word embeddings
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

sports_embeddings_256 = sentence_model.encode(docs, convert_to_numpy=True)

np.save('/content/drive/MyDrive/sports_embeddings_256', sports_embeddings_256)

In [17]:
# load sentence data set 
transcripts = pd.read_csv('/content/drive/MyDrive/sport_sent_1.csv.gz', compression='gzip', usecols=[2])
print(transcripts.shape)
print(transcripts.head(2))

docs = transcripts.transcript_subset.to_list()
print(len(docs))

# Sentence embeddings
sports_embeddings_sentence_1 = sentence_model.encode(docs, convert_to_numpy=True)

np.save('/content/drive/MyDrive/sports_embeddings_sentence_1', sports_embeddings_sentence_1)

(500000, 1)
  transcript_subset
0          Awesome.
1      I like that.
500000


In [ ]:
# Initialize and rescale T-SVD embeddings and dim reduce using t-SNE
#from sklearn.decomposition import TruncatedSVD
#from sklearn.manifold import TSNE

T_SNE = TSNE(n_components=3)
tsvd_embeddings = TruncatedSVD(n_components=50).fit_transform(embeddings_256)
tsne_dimreduced_embeddings = T_SNE.fit_transform(X=tsvd_embeddings)
np.save('/content/drive/MyDrive/tsne_dimred_embeddings', tsne_dimreduced_embeddings)